# Modeling of a Torsional Spring in Finesse

This notebooks adds the pendulum dynamics of the mirrors to a simple Fabry Perot cavity, computes the optical response to angular motion, and computes the effects of radiation pressure on the mirror dynamics.

[__1.__](#model) Model definition

[__2.__](#frequency-response) Compute frequency response

[__3.__](#optical-response) Optical response

[__4.__](#mechanical-response) Radiation pressure modification of the mechanical response

The OptickleTorsionalSpring notebook goes through the identical calculations with Optickle.

In [ ]:
import numpy as np
import qlance.finesse as fin
import qlance.controls as ctrl
import qlance.filters as filt
import pykat
from qlance.plotting import plotTF
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
mpl.rc('figure', figsize=(8, 11))

mpl.rcParams.update({'text.usetex': False,
                     'mathtext.fontset': 'cm',
                     'lines.linewidth': 3,
                     'lines.markersize': 10,
                     'font.size': 16,
                     'axes.grid': True,
                     'grid.alpha': 0.5,
                     'legend.loc': 'best',
                     'savefig.dpi': 80,
                     'pdf.compression': 9})

<a name="model"> </a>

## Model Definition

See BasicFinesseFP for an introduction to Finesse model building.

Note that when adding mirrors to a Finesse model with QLANCE's model building functions, the radius of curvature is always the physical radius of curvature of the mirror. Optical cavities built with classic Finesse code often have the sign of the radius of curvature of the input coupler opposite to its physical sign depending on the order components were added to the model. There is never an ambiguity when using QLANCE's functions.

In [ ]:
fmod = 11e3  # modulation frequency [Hz]
gmod = 0.1   # modulation depth
Pin = 10e3   # input power [W]
Ti = 0.014   # ITM transmissivity
Lcav = 40e3  # cavity length [m]
Ri = 34e3    # ITM radius of curvature [m]
Re = 36e3    # ETM radius of curvature [m]

# Mechanical response of the torsional pendulum
I = 25   # moment of inertia of the mirror [kg m^2]
f0 = 1   # resonance frequency [Hz]
Q = 100  # Q factor
poles = np.array(filt.resRoots(2*np.pi*f0, Q, Hz=False))

kat = pykat.finesse.kat()

# make the cavity
fin.addMirror(kat, 'EX', Chr=1/Re)
fin.addMirror(kat, 'IX', Thr=Ti, Chr=1/Ri)
fin.addSpace(kat, 'IX_fr', 'EX_fr', Lcav)
fin.setCavityBasis(kat, 'IX_fr', 'EX_fr')

# set the pitch response by giving a zpk definition of the dynamics
fin.setMechTF(kat, 'EX', [], poles, 1/I, doftype='pitch')
fin.setMechTF(kat, 'IX', [], poles, 1/I, doftype='pitch')

# add input
fin.addLaser(kat, 'Laser', Pin)
fin.addModulator(kat, 'Mod', fmod, gmod, 1, 'pm')  # RF modulator for PDH sensing
fin.addSpace(kat, 'Laser_out', 'Mod_in', 0)
fin.addSpace(kat, 'Mod_out', 'IX_bk', 0)

# add DC and RF photodiodes
fin.addReadout(kat, 'REFL', 'IX_bk', fmod, 0, doftype='pitch')

fin.monitorMotion(kat, 'EX', doftype='pitch')
fin.monitorMotion(kat, 'IX', doftype='pitch')

kat.phase = 2   # always use phase 2 when doing anything with HOMs!
kat.maxtem = 1  # use up to 1st order HOMs

The free mechanical response of the pendulum must be defined to compute the effects of radiation pressure. The command 
```python
fin.setMechTF(kat, name, zs, ps, k, doftype)
```
sets this response for the optic called `name` to a mechanical plant specified by zeros, poles, and a gain. The convenience function `resRoots(omega0, Q)` computes the conjugate poles (or zeros) of a resonance with a given frequency and Q factor.

*Note for Finesse users: Unlike specifying a plant directly with classic Finesse code where only half of the complex zeros and poles should be given, QLANCE uses the zpk model as defined and will give an error if an unphysical plant is given.*

When working with higher order modes, a Hermite-Gauss basis must be defined. The simplest way to do this is to have Finesse compute it by using the basis of a stable cavity. The command `setCavityBasis(kat, node1, node2)` tells Finesse to use the cavity formed by `node1` and `node2` to define the basis. Multiple cavities can be used to define the basis in more complicated models.

You should also always set `kat.phase = 2` otherwise the simulation may give unphysical results.

<a name="frequency-response"> </a>

## Frequency Response

See BasicFinesseFP for an overview of frequency response calculations. Since we are interested in the pitch dynamics here we have to run the model with the `doftype` (degree of freedom) keyword:
```python
katFR.run(fmin, fmax, npts, doftype='pitch')
```

In [ ]:
# make a frequency response object from the finesse model
katFR = fin.KatFR(kat)

In [ ]:
# compute the AC response matrix, i.e. the optomechanical plant
fmin = 1e-1
fmax = 30
npts = 1000
katFR.run(fmin, fmax, npts, doftype='pitch')

Transfer functions which are linear combinations of drives can also be computed directly by specifying those combinations as a dictionary. In this case we'll look at the hard $\theta_\mathrm{h}$ and soft $\theta_\mathrm{s}$ modes defined as
$$\begin{bmatrix}
\theta_\mathrm{s}\\
\theta_\mathrm{h}
\end{bmatrix}
=
\begin{bmatrix}
r & 1 \\
-1 & r
\end{bmatrix}
\begin{bmatrix}
\theta_\mathrm{i}\\
\theta_\mathrm{e}
\end{bmatrix}, \qquad
r = \frac{2}{(g_\mathrm{i} - g_\mathrm{e}) + \sqrt{(g_\mathrm{i} - g_\mathrm{e})^2 + 4}}, \qquad
g_{\mathrm{e}(\mathrm{i})} = 1 - \frac{L}{R_{\mathrm{e}(\mathrm{i})}}
$$

In [ ]:
gi = 1 - Lcav/Ri
ge = 1 - Lcav/Re
r = 2/((gi - ge) + np.sqrt((gi - ge)**2 + 4))

HARD = dict(IX=-1, EX=r)
SOFT = dict(IX=r, EX=1)

As described in BasicFinesseFP `DegreeOfFreedom` instances can also be used

In [ ]:
EX = ctrl.DegreeOfFreedom('EX', doftype='pitch')

<a name="optical-response"> </a>

### Optical response

See BasicFinesseFP for an overview of calculating transfer functions.

In [ ]:
# transfer functions can be computed directly
tf_REFL_I_SOFT = katFR.getTF('REFL_I', SOFT, doftype='pitch')

In [ ]:
# plotTF is a convenience function to quickly plot a transfer function
fig = katFR.plotTF('REFL_I', HARD, doftype='pitch', label='Hard');
plotTF(katFR.ff, tf_REFL_I_SOFT, *fig.axes, label='Soft');
katFR.plotTF('REFL_I', EX, *fig.axes, label='EX')  # doftype already specified in DegreeOfFreedom
fig.axes[0].legend();
fig.axes[0].set_title('Optical Response');
fig.axes[0].set_ylabel('Magnitude [W/rad]');

<a name="mechanical-response"> </a>


### Radiation pressure modification of the mechanical response

The radiation pressure stiffens the hard mode increasing the resonance frequency from that of the free pendulum and softens the soft mode decreasing the resonance frequency.

```python
katFR.getMechTF(drive_to, drive_from, dof)
```
calculates the mechanical response of `drive_to` due to forces or torques on `drive_from`. As with `getTF`, the drives can be either strings for a single drive or a dictionaries specifying linear combinations of drives.

In [ ]:
# For comparisson, define a filter showing the free pendulum response
# More on filters in the control systems example. This is just for plotting here
pend = filt.ZPKFilter([], poles, 1/I, Hz=False)

In [ ]:
# Again, mechanical transfer functions can be computed directly
tf_mech_SOFT = katFR.getMechTF(SOFT, SOFT, doftype='pitch')

In [ ]:
# and plotMechTF is a convenience function
fig = katFR.plotMechTF(HARD, HARD, doftype='pitch', label='Hard')
plotTF(katFR.ff, tf_mech_SOFT, fig.axes[0], fig.axes[1], label='Soft')
katFR.plotMechTF(EX, EX, *fig.axes, label='EX')  # doftype already specified in DegreeOfFreedom
pend.plotFilter(katFR.ff, *fig.axes, label='Free pendulum')
fig.axes[0].legend();
fig.axes[0].set_title('Mechanical Response');
fig.axes[0].set_ylabel('Magnitude [rad/(N m)]');